In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 92kB/s 
     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 6.7MB 22.9MB/s 
     |████████████████████████████████| 983kB 30.8MB/s 
     |████████████████████████████████| 245kB 46.0MB/s 
     |████████████████████████████████| 675kB 32.7MB/s 
     |████████████████████████████████| 471kB 39.4MB/s 
     |████████████████████████████████| 450kB 38.9MB/s 
     |████████████████████████████████| 3.9MB 28.2MB/s 
     |████████████████████████████████| 890kB 30.3MB/s 
     |████████████████████████████████| 1.1MB 32.7MB/s 
     |████████████████████████████████| 3.8MB 33.1MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [27]:
train_b=pd.read_csv("/content/drive/My Drive/minor/hindi_dataset/hindi_dataset.tsv",sep='\t',encoding='utf-8') #training data
test_b=pd.read_csv("/content/drive/My Drive/minor/hindi_dataset/hasoc2019_hi_test_gold_2919.tsv",sep='\t',encoding='utf-8') #testing data
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_1'],test_size=0.15,random_state=42) #training and validation split
target_names=['HOF','NOT'] #names of classes

In [6]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [7]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-multilingual-cased' #bert predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for bert
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get bert classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for bert

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: hi
train sequence lengths:
	mean : 28
	95percentile : 56
	99percentile : 61


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:520: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: hi
test sequence lengths:
	mean : 28
	95percentile : 57
	99percentile : 62


In [8]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 661 steps, validate for 22 steps
Epoch 1/4
661/661 [==============================] - 333s 503ms/step - loss: 0.6381 - accuracy: 0.6383 - val_loss: 0.6283 - val_accuracy: 0.6543
Epoch 2/4
661/661 [==============================] - 312s 471ms/step - loss: 0.6263 - accuracy: 0.6381 - val_loss: 0.6225 - val_accuracy: 0.6671
Epoch 3/4
661/661 [==============================] - 311s 471ms/step - loss: 0.6728 - accuracy: 0.5864 - val_loss: 0.6752 - val_accuracy: 0.6029
Epoch 4/4
661/661 [==============================] - 311s 471ms/step - loss: 0.6576 - accuracy: 0.6267 - val_loss: 0.6475 - val_accuracy: 0.6243


In [9]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.67      0.61      0.64       380
           1       0.58      0.64      0.61       320

    accuracy                           0.62       700
   macro avg       0.62      0.63      0.62       700
weighted avg       0.63      0.62      0.62       700



array([[231, 149],
       [114, 206]])

In [10]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  bert

In [33]:
predictor.get_classes()

['HOF', 'NOT']

In [28]:
test_b_text=list(test_b['text']) #get text for testing
test_b['task_1']=test_b['task_1'].map({'HOF':0,'NOT':1})
test_b_label=list(test_b['task_1']) #get label of testing
# val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data

In [29]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [30]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [31]:
from sklearn.metrics import classification_report

In [32]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.58      0.55      0.56       605
           1       0.63      0.65      0.64       713

    accuracy                           0.61      1318
   macro avg       0.60      0.60      0.60      1318
weighted avg       0.61      0.61      0.61      1318

